# Example queries for Case Counts on COVID-19 Knowledge Graph
[Work in progress]

This notebook demonstrates how to run Cypher queries in a Jupyter Notebook by connecting to a database server.

In [1]:
import datetime
import pandas as pd
from py2neo import Graph

In [2]:
pd.options.display.max_rows = None  # display all rows
pd.options.display.max_columns = None  # display all columsns

In [3]:
graph = Graph("bolt://132.249.238.185:7687", user="reader", password="demo")

In [4]:
# currently defining "yesterday" as two days from current date, 
# since there are time periods during the day, where data from the 
# previous day are not yet available in UTC date.
today = datetime.datetime.utcnow().date()
yesterday = today - datetime.timedelta(days=2)

## COVID Case Data

### Case counts by Country

In [5]:
query = """
MATCH (c:Cases{date: date($day), source: 'JHU'})-[:REPORTED_IN]->(n:Country)
RETURN n.name, c.cummulativeConfirmed, c.cummulativeDeaths, c.date as dateUTC
ORDER by n.name
"""
graph.run(query, day=yesterday).to_data_frame()

,n.name,c.cummulativeConfirmed,c.cummulativeDeaths,dateUTC
0,Afghanistan,27532,546,2020-06-18
1,Albania,1788,39,2020-06-18
2,Algeria,11385,811,2020-06-18
3,Andorra,855,52,2020-06-18
4,Angola,166,8,2020-06-18
5,Anguilla,3,0,2020-06-18
6,Antigua and Barbuda,26,3,2020-06-18
7,Argentina,37510,948,2020-06-18
8,Armenia,18698,309,2020-06-18
9,Aruba,101,3,2020-06-18


### Case counts by US States aggregated from US Counties
Note, some counties in the Johns Hopkins dataset cannot be mapped to US counties. They are listed in as "None"

In [6]:
query = """
MATCH (c:Cases{date: $date, source: 'JHU'})-[:REPORTED_IN]->(a:Admin2)-[:IN]->(a1:Admin1)
RETURN a1.name as state, a1.code, a1.location, sum(c.cummulativeDeaths) as deaths, sum(c.cummulativeConfirmed) as confirmed, c.date as dateUTC
ORDER BY a1.code
"""
graph.run(query, date=yesterday).to_data_frame()

,state,a1.code,a1.location,deaths,confirmed,dateUTC
0,Alaska,AK,"(-150.00028, 64.00028)",12,704,2020-06-18
1,Alabama,AL,"(-86.75026, 32.75041)",799,27614,2020-06-18
2,Arkansas,AR,"(-92.50044, 34.75037)",205,12712,2020-06-18
3,Arizona,AZ,"(-111.50098, 34.5003)",1282,43445,2020-06-18
4,California,CA,"(-119.75126, 37.25022)",5308,164066,2020-06-18
5,Colorado,CO,"(-105.50083, 39.00027)",1638,29886,2020-06-18
6,Connecticut,CT,"(-72.66648, 41.66704)",4226,45170,2020-06-18
7,Delaware,DE,"(-75.49992, 39.00039)",431,10485,2020-06-18
8,Florida,FL,"(-82.5001, 28.75054)",3014,84394,2020-06-18
9,Georgia,GA,"(-83.50018, 32.75042)",2564,56893,2020-06-18


### Current cases in San Diego County

In [7]:
admin2 = 'San Diego County'

query = """
MATCH (c:Cases{date: date($day), source: 'JHU'})-[:REPORTED_IN]->(a:Admin2{name: $admin2})
RETURN a.name as name, c.cummulativeConfirmed as confirmed, c.cummulativeDeaths as deaths, c.date as dateUTC
"""
graph.run(query, admin2=admin2, day=yesterday).to_data_frame()

,name,confirmed,deaths,dateUTC
0,San Diego County,10092,331,2020-06-18


### Cases in San Diego County time series

In [8]:
query = """
MATCH (c:Cases{source: 'JHU'})-[:REPORTED_IN]->(a:Admin2{name: $admin2})
RETURN a.name as name, c.cummulativeConfirmed as confirmed, c.cummulativeDeaths as deaths, c.date as dateUTC
ORDER BY c.date DESC
"""
graph.run(query, admin2=admin2).to_data_frame()

,name,confirmed,deaths,dateUTC
0,San Diego County,10350,332,2020-06-19
1,San Diego County,10092,331,2020-06-18
2,San Diego County,9730,323,2020-06-17
3,San Diego County,9730,323,2020-06-16
4,San Diego County,9610,320,2020-06-15
5,San Diego County,9440,319,2020-06-14
6,San Diego County,9314,319,2020-06-13
7,San Diego County,8998,308,2020-06-12
8,San Diego County,8998,308,2020-06-11
9,San Diego County,8837,305,2020-06-10


### Current cases in San Diego Country by Zip code

In [9]:
query = """
MATCH (c:Cases{date: date($day), source: 'SDHHSA'})-[:REPORTED_IN]->(p:PostalCode)
RETURN p.name as zip, c.cummulativeConfirmed as confirmed, c.date as date
ORDER by zip
"""
graph.run(query, day=yesterday).to_data_frame()

,zip,confirmed,date
0,91901,18,2020-06-18
1,91902,53,2020-06-18
2,91905,4,2020-06-18
3,91906,6,2020-06-18
4,91910,493,2020-06-18
5,91911,667,2020-06-18
6,91913,189,2020-06-18
7,91914,59,2020-06-18
8,91915,102,2020-06-18
9,91916,3,2020-06-18
